## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-06-19-08-19 +0000,nypost,"Female attacker pummels woman, 72, over seat o...",https://nypost.com/2026/02/06/us-news/female-a...
1,2026-02-06-19-07-47 +0000,nyt,"As Olympics Open, Pope Warns Against Using Spo...",https://www.nytimes.com/2026/02/06/world/europ...
2,2026-02-06-19-07-12 +0000,nyt,Who will light the Olympic cauldrons?,https://www.nytimes.com/live/2026/02/06/world/...
3,2026-02-06-19-02-00 +0000,wsj,President Trump posted a video including image...,https://www.wsj.com/politics/trump-obamas-apes...
4,2026-02-06-19-01-37 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/06/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2523/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,34
200,epstein,21
69,new,16
9,olympics,14
306,talks,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
322,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...,100
331,2026-02-05-22-22-03 +0000,nypost,Hillary Clinton demands public hearing on Epst...,https://nypost.com/2026/02/05/us-news/hillary-...,81
278,2026-02-06-01-19-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,81
3,2026-02-06-19-02-00 +0000,wsj,President Trump posted a video including image...,https://www.wsj.com/politics/trump-obamas-apes...,77
111,2026-02-06-16-02-27 +0000,latimes,"Trump shares, then deletes, racist image of th...",https://www.latimes.com/politics/story/2026-02...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
322,100,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...
7,61,2026-02-06-19-00-00 +0000,wsj,Bill and Hillary Clinton are once again prepar...,https://www.wsj.com/politics/looming-epstein-d...
235,56,2026-02-06-09-04-24 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...
85,53,2026-02-06-17-01-33 +0000,wapo,U.S. and Iran hold nuclear talks amid threats ...,https://www.washingtonpost.com/national-securi...
262,47,2026-02-06-04-26-26 +0000,nypost,AOC touts money for her NYC district in fundin...,https://nypost.com/2026/02/05/us-news/aoc-tout...
16,45,2026-02-06-18-49-54 +0000,nypost,Camera snatched off Nancy Guthrie’s house coul...,https://nypost.com/2026/02/06/us-news/camera-s...
89,43,2026-02-06-16-54-39 +0000,nyt,Russian General Is Shot in Latest Attack on a ...,https://www.nytimes.com/2026/02/06/world/europ...
42,42,2026-02-06-17-58-09 +0000,nypost,Mother of young kids identified as victim in h...,https://nypost.com/2026/02/06/us-news/mother-o...
57,40,2026-02-06-17-42-56 +0000,nypost,Daughter of woman allegedly killed by Jill Bid...,https://nypost.com/2026/02/06/us-news/daughter...
126,40,2026-02-06-15-11-52 +0000,bbc,Amazon shares fall as it joins Big Tech AI spe...,https://www.bbc.com/news/articles/c150e144we3o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
